# ONIOM in pySCF

## Introduction to ONIOM
This notebook is intended to provide an overview of our implementation of the ONIOM method (https://pubs.acs.org/doi/10.1021/cr5004419), a hybrid QM/MM technique. The ONIOM method enables the user to leverage the precision of expensive computational chemistry tools, only where expressly necessary. 

When studying large molecules, and their interactions with other systems, use of expensive computational techniques is often not only intractable, but also unnecessary. For example, when considering the interaction between a water molecule and a functional group appended to a large graphite flake, it is not reasonable to attempt to treat this entire problem at the level of something like CCSD. In such situations, the ONIOM method enables us to target the fragments of the problem at hand with the most sophisticated techniques we can afford, while relegating the rest of the problem to the domain of more affordable computational methods. This method can become particularly advantageous when we are interested in energy differences, where the contribution to the total molecular energy from our relatively inert volumes become negligible. 

Formally, the energy evaluated by ONIOM is expressed as:

\begin{equation}
E_{ONIOM} = E_0^{LOW} + \sum_{i=1}^N (E_i^{HIGH} - E_i^{LOW})
\end{equation}


The general procedure for ONIOM is as follows. The user identifies a system of interest. A low-cost method is used to compute the total energy of the system. Subsequently, a subset of the molecule is defined as a model fragment. The fragment is isolated, and its energy is computed using, both the low-cost method as above, and a high-cost method which can be applied on this reduced system. The difference in energy between these two models is then added to our total energy. In this way, we can interpret the ONIOM method as an iterative procedure where the error associated with our low-cost solver is removed. So long as our solvers obey the variational principle, the equation above indicates that the ground state energy achieved via ONIOM will always be between the energy one expects if solving the entire system with the high-cost method, and that of the low-cost method.

The equation above is formulated to allow us to expand ONIOM beyond a single fragment -- in principle, many such fragments can be defined to progressively improve upon our result. This may apply to the situation where we have more than one active site on our large molecule, or where an incremental strategy can be utilized to further mitigate against errors associated with using our low-cost method in the vicinity of our active region.


## Motivation
The ONIOM method has been implemented in software such as Gaussian, and nwChem. Presently, we will discuss our implementation of this method in pySCF. The reasons for this development are several. First of all, this gives us the opportunity to tailor the implementation to our own needs, an obvious advantage, when the cost of development is not substantial. Far more importantly, we intend to integrate quantum methods into the ONIOM pipeline, specifically, through introduction of our VQE solvers. As pySCF is the preferred chemistry backend of choice for existing QDK's like openfermion and Qiskit, an implementation of ONIOM within this environment will enable a natural extension to quantum algorithms.

## Code Overview

In what follows, we are going to describe our implementation of ONIOM, as well as our developments to enable use of ONIOM as a solver for geometric optimization, using pySCF's connection to the geomeTRIC software package. As we will see, the form of the ONIOM energy is easily amenable to geometric optimization. Before getting into the details of our code, let's consider a simple example of the use of ONIOM. We'll consider an example worked out for nwChem's ONIOM implementation, as a test of our procedure. This is going to be CH$_3$-CH$_2$. This is a reaction product formed during the decomposition of CH$_3$-CH$_3$ $\rightarrow$ CH$_2$-CH$_3$ + H.

In [1]:
from pyscf import mp,scf
from utility import link,fragment
from utility import load_text,build_molecule
import oniom_v3 as oniom

In [11]:
atoms = load_text('CH2CH3.xyz')

spin = 1
basis = '6311++g**'

high = {'method':'mp2','basis':basis}
low = {'method':'uhf','basis':basis}

We're going to break the system at the C-C bond, and model the CH$_2$ fragment at the level of MP2, while the system as a whole will be solved with the less expensive Hartree Fock.

In [12]:
links = [link(3,7,0.709)]
system = fragment(low,spin=1,charge=0)
model = fragment(solver=[high,low],Natoms=3,links=links,spin=1,charge=0)
fragments = [system,model]

In this last step, we've specified the system and the model fragments. We detail the solver and basis to be used, and the bonds, or links, that are broken in forming the model fragment. This is an important step. In defining the model fragment, it is important that the charge, and spin of the system remain the same as when appended to the larger system. When a bond is broken, we then add a link atom to the severed bond, to correct for this. This is the same as the hydrogenation of dangling bonds which is routinely performed in computational chemistry. For each broken link, we specify the indices of the atoms (from our structure file) for which the bond is broken, and a scale factor for the bond length. For C-C bonds, a value of ~70% works well. We break the bond, and place a hydrogen atom at 0.709$\times$ the distance of the original bond length. When breaking bonds other than single-bonds, a species other than H can be specified to correct for this. We'll come back to this when we look at the *link* class in greater detail.

In [13]:
oniom_model = oniom.oniom_model(atoms,fragments)
e_tot = oniom_model.run()
print('ONIOM Energy:',e_tot)

converged SCF energy = -39.5731618458956  <S^2> = 0.76088911  2S+1 = 2.0108596
converged SCF energy = -78.6201415763761  <S^2> = 0.76314827  2S+1 = 2.0131053
E(UMP2) = -39.7265568017675  E_corr = -0.153394955871856
converged SCF energy = -39.5731618458956  <S^2> = 0.76088911  2S+1 = 2.0108596
ONIOM Energy: -78.77353653224797


Finally, we instantiate, and run the ONIOM solver. For a total energy, we get a value of -78.7730 Ha. We can compare this to the results achieved with using solely our low-cost, or high-cost method, to get a sense of the improvement.

In [14]:
full = build_molecule(atoms,basis,spin=spin)
mf = scf.UHF(full).run()
fullsolve = mp.MP2(mf,frozen=1)
fullsolve.run()

converged SCF energy = -78.6201415763761  <S^2> = 0.76314827  2S+1 = 2.0131053
E(UMP2) = -78.9096549806102  E_corr = -0.289513404234034


In [15]:
print('ONIOM Energy: ',e_tot)
print('Full HF Energy: ',mf.e_tot)
print('Full MP2 Energy: ',fullsolve.e_tot)

ONIOM Energy:  -78.77353653224797
Full HF Energy:  -78.62014157637613
Full MP2 Energy:  -78.90965498061016


Great, so we see then that ONIOM has given us a lower ground state energy for our molecule than HF could have given us, although not quite as good as what we could have got from running MP2 on our system as a whole. Interestingly, this particular case is almost exactly half-way between the two. In this example, we have utilized different electronic structure solvers (MP2 and HF) to perform the ONIOM calculation. More generally, we note that one can use the same solver, with different basis choices (e.g. STO-3g and 6311++g**). The ONIOM method provides a great deal of flexibility in this manner. Looking forward, we intend to implement VQE as a solver in our ONIOM framework. 

## Overview of Code Structure

In our ONIOM implementation, there are three important classes that require some description. The centre of an ONIOM calculation is, unsurprisingly the *oniom_model* class. This keeps track of the system geometry, the various solvers, and how they come together in evaluating the total energy. While *oniom_model* provides organizational structure to the calculation, the *layer* class is the heart of our code. Once the various fragments with their geometries, basis, and solver methods have been specified, there is no need for an hierarchy to be maintained. We simply keep track of the sign with which a layer's energy should be added to the total energy. In the above example, we have three layers: 

1. DFT on CH$_2$-H (+)
2. HF on  CH$_2$-H (-)
3. HF on  CH$_2$-CH$_3$ (+)

This structure becomes particularly useful when we consider geometric optimization with ONIOM. In order to perform such optimization, we must have access to the energy gradients. The ONIOM gradient is defined in, for example, https://doi.org/10.1021/cr5004419 and is straightforward to implement. The gradient is defined with respect to atomic coordinates of the system, R = R$_{SYS}$.
\begin{equation}
\frac{dE_{ONIOM}}{dR} = \frac{dE_{LOW}[R]}{dR} + \sum_i \frac{dE_{HIGH_i}[R_i]}{dR} - \frac{dE_{LOW_i}[R_i]}{dR}
\end{equation}

One can use chain rule to express 
\begin{equation}
        \frac{d}{dR} = \frac{dR_i}{dR} \frac{d}{dR_i} = J(R_i,R) \frac{d}{dR_i}
\end{equation}
\begin{equation}
\frac{dE_{ONIOM}}{dR} = \frac{dE_{LOW}[R]}{dR} + \sum_i \frac{dE_{HIGH_i}[R_i]}{dR_i} \cdot J(R_i,R) - \frac{dE_{LOW_i}[R_i]}{dR_i} \cdot J(R_i,R)
\end{equation}

In practical terms however, utilizing the *layer* class, this expression simplifies to

\begin{equation}
\frac{dE_{ONIOM}}{dR} =  \sum_i \mathrm{sgn}[i]\frac{dE_{i}[R_i]}{dR_i} \cdot J(R_i,R)
\end{equation}



In practice then, each layer is instantiated with its Jacobian, making this a trivial calculation done 
using numpy's einsum method. This conveniently handles the different number of atoms in models, system,
relying on the well-defined functional form of the link-atoms placement to enable proper treatment
of all atoms in the system. Furthermore, as the layer class keeps track of broken links and their rescaling factors, we can perform geometric optimization on the whole system, updating each fragment's specific geometry at each iteration.

When we execute an update to the system geometry, the position coordinates of each fragment are updated with marginal overhead.

To enable integration with pySCF's geometric optimizer tools, we have implemented the energy and gradient computations as *scanners*, as done for other solvers in pySCF's library. This is detailed in *grad.py*. 